**Transfer Learning Approach Using Resnet for Emotion Classification**
 - ref: https://github.com/keras-team/keras-applications/blob/master/keras_applications/resnet50.py 

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import cv2
import glob
from time import time
import pandas as pd

import numpy as np
import keras
from keras.utils import Sequence
from keras.models import Sequential, Model 
from keras import layers
from keras.callbacks import TensorBoard
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
import os
import tensorflow as tf
from keras import Input, layers,models
from keras.layers import Dense, Flatten, Conv3D, MaxPooling3D

Using TensorFlow backend.


In [0]:
# path = "/content/drive/My Drive/Affective Computing/Assn_3/"

In [0]:
def identity_block(input_tensor, kernel_size, filters, stage, block):
    filters1, filters2, filters3 = filters
    bn_axis = 3
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'

    x = layers.Conv2D(filters1, (1, 1),
                      kernel_initializer='he_normal',
                      name=conv_name_base + '2a')(input_tensor)
    x = layers.BatchNormalization(axis=bn_axis, name=bn_name_base + '2a')(x)
    x = layers.Activation('relu')(x)

    x = layers.Conv2D(filters2, kernel_size,
                      padding='same',
                      kernel_initializer='he_normal',
                      name=conv_name_base + '2b')(x)
    x = layers.BatchNormalization(axis=bn_axis, name=bn_name_base + '2b')(x)
    x = layers.Activation('relu')(x)

    x = layers.Conv2D(filters3, (1, 1),
                      kernel_initializer='he_normal',
                      name=conv_name_base + '2c')(x)
    x = layers.BatchNormalization(axis=bn_axis, name=bn_name_base + '2c')(x)

    x = layers.add([x, input_tensor])
    x = layers.Activation('relu')(x)
    return x


def conv_block(input_tensor,
               kernel_size,
               filters,
               stage,
               block,
               strides=(2, 2)):
    filters1, filters2, filters3 = filters
    bn_axis = 3
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'

    x = layers.Conv2D(filters1, (1, 1), strides=strides,
                      kernel_initializer='he_normal',
                      name=conv_name_base + '2a')(input_tensor)
    x = layers.BatchNormalization(axis=bn_axis, name=bn_name_base + '2a')(x)
    x = layers.Activation('relu')(x)

    x = layers.Conv2D(filters2, kernel_size, padding='same',
                      kernel_initializer='he_normal',
                      name=conv_name_base + '2b')(x)
    x = layers.BatchNormalization(axis=bn_axis, name=bn_name_base + '2b')(x)
    x = layers.Activation('relu')(x)

    x = layers.Conv2D(filters3, (1, 1),
                      kernel_initializer='he_normal',
                      name=conv_name_base + '2c')(x)
    x = layers.BatchNormalization(axis=bn_axis, name=bn_name_base + '2c')(x)

    shortcut = layers.Conv2D(filters3, (1, 1), strides=strides,
                             kernel_initializer='he_normal',
                             name=conv_name_base + '1')(input_tensor)
    shortcut = layers.BatchNormalization(
        axis=bn_axis, name=bn_name_base + '1')(shortcut)

    x = layers.add([x, shortcut])
    x = layers.Activation('relu')(x)
    return x


def ResNet50():



    # Determine proper input shape
    input_shape = (224,224,3)
    img_input = (224,224,3)
    img_input = Input(shape=input_shape)
    bn_axis = 3

    x = layers.ZeroPadding2D(padding=(3, 3), name='conv1_pad')(img_input)
    x = layers.Conv2D(64, (7, 7),
                      strides=(2, 2),
                      padding='valid',
                      kernel_initializer='he_normal',
                      name='conv1')(x)
    x = layers.BatchNormalization(axis=bn_axis, name='bn_conv1')(x)
    x = layers.Activation('relu')(x)
    x = layers.ZeroPadding2D(padding=(1, 1), name='pool1_pad')(x)
    x = layers.MaxPooling2D((3, 3), strides=(2, 2))(x)

    x = conv_block(x, 3, [64, 64, 256], stage=2, block='a', strides=(1, 1))
    x = identity_block(x, 3, [64, 64, 256], stage=2, block='b')
    x = identity_block(x, 3, [64, 64, 256], stage=2, block='c')

    x = conv_block(x, 3, [128, 128, 512], stage=3, block='a')
    x = identity_block(x, 3, [128, 128, 512], stage=3, block='b')
    x = identity_block(x, 3, [128, 128, 512], stage=3, block='c')
    x = identity_block(x, 3, [128, 128, 512], stage=3, block='d')

    x = conv_block(x, 3, [256, 256, 1024], stage=4, block='a')
    x = identity_block(x, 3, [256, 256, 1024], stage=4, block='b')
    x = identity_block(x, 3, [256, 256, 1024], stage=4, block='c')
    x = identity_block(x, 3, [256, 256, 1024], stage=4, block='d')
    x = identity_block(x, 3, [256, 256, 1024], stage=4, block='e')
    x = identity_block(x, 3, [256, 256, 1024], stage=4, block='f')

    x = conv_block(x, 3, [512, 512, 2048], stage=5, block='a')
    x = identity_block(x, 3, [512, 512, 2048], stage=5, block='b')
    x = identity_block(x, 3, [512, 512, 2048], stage=5, block='c')
    x = layers.GlobalMaxPooling2D()(x)

    inputs = img_input
    # Create model.
    model = models.Model(inputs, x, name='resnet50')

    # Load weights.
    #model.load_weights(weights)

    return model

def resnet50_emotion_recoginition():  
    # weights = 'path of h5 file'
    facial_model = ResNet50()
    x = facial_model.output
    x = layers.Dense(64)(x)
    x = layers.Activation('relu')(x)
    x = layers.Dropout(0.5)(x)
    x = layers.Dense(7)(x)
    x = layers.Activation('softmax')(x) 

    model = models.Model(facial_model.input,x,name='resnet50_emotion_recoginition')
    return model

In [4]:
emotion_recoginiser = resnet50_emotion_recoginition()
emotion_recoginiser.summary()

Model: "resnet50_emotion_recoginition"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 112, 112, 64) 256         conv1[0][0]                      
______________________________________________________________________

In [0]:
# class for inputing data batch by batch
class MY_Generator(Sequence):
    def __init__(self, video_filenames, video_gt, batch_size):
        self.video_filenames, self.video_gt = video_filenames, video_gt
        self.batch_size = batch_size

    def __len__(self):
        return (len(self.video_filenames))//self.batch_size 

    def __getitem__(self, idx):
        batch_x = self.video_filenames[idx:idx+self.batch_size]
        batch_y = self.video_gt[idx:idx+self.batch_size]
        
        video = np.zeros((self.batch_size,224,224,3))
        # y = cv2.imread(batch_y[0])
        for i in range(self.batch_size):
            x = load_img(batch_x[i])
            x = img_to_array(x)
            x = cv2.resize(x,(224,224))
            video[i,:,:,:] = x[:,:,:]
        video /= 127.5
        video -= 1.0 
        gt = np.zeros((self.batch_size,7))
        for i in range(self.batch_size):
          label = batch_y[i]
          if label == 'happy':
            gt[i,:] = [0,0,0,0,0,0,0]
          elif label == 'sad':
            gt[i,:] = [1,0,0,0,0,0,0]
          elif label == 'disgust':
            gt[i,:] = [0,1,0,0,0,0,0]
          elif label == 'angry':
            gt[i,:] = [0,0,1,0,0,0,0]
          elif label == 'surprised':
            gt[i,: ] = [0,0,0,1,0,0,0]
          elif label == 'calm':
            gt[i,:] = [0,0,0,0,1,0,0]
          elif  label == 'neutral':
            gt[i,:] = [0,0,0,0,0,1,0]
          else:
            gt[i,:] = [0,0,0,0,0,0,1]
          # print(image,gt)
        return video,gt

In [6]:
def main():
  #Loading the architecture
  model = resnet50_emotion_recoginition()
  #lo
  model.load_weights('/content/drive/My Drive/Affective Computing/Assignment 3/my_data/resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5',by_name=True)
  #for layer in model.layers[:135]:
  #  layer.trainable = False  
  adam = keras.optimizers.Adam(lr=0.001)
  model.compile(loss=keras.losses.categorical_crossentropy, optimizer=adam, metrics=['accuracy'])
  csv_file = pd.read_csv('/content/drive/My Drive/Affective Computing/Assignment 3/my_data/TRAIN.csv')
  z = csv_file.iloc[:,1].values
  gt = csv_file.iloc[:,2].values
  final_index = len(glob.glob('/content/drive/My Drive/Affective Computing/Assignment 3/my_data/TRAIN.csv'))
  print(final_index)

  csv_test_file = pd.read_csv('/content/drive/My Drive/Affective Computing/Assignment 3/my_data/TEST.csv')
  test_z = csv_test_file.iloc[:,1].values
  gt_test = csv_test_file.iloc[:,2].values
  test_final_index = len(glob.glob('/content/drive/My Drive/Affective Computing/Assignment 3/my_data/TEST.csv'))

  initial_index = 1
  final_index = 10
  train_video_filenames = []
  train_video_filenames[:] = z[:]
  train_video_ground_truth = []
  train_video_ground_truth = gt[:]
  for i in range(len(train_video_filenames)):
    train_video_filenames[i] = train_video_filenames[i][:]
  

  validate_video_filenames = []
  validate_video_filenames[:] = test_z[:]
  for i in range(len(validate_video_filenames)):
    validate_video_filenames[i] = validate_video_filenames[i][:]

  validate_video_ground_truth = []
  validate_video_ground_truth = gt_test[:]

  #creating my_generator objects
  batch_size = 16
  my_training_batch_generator = MY_Generator(train_video_filenames,train_video_ground_truth, batch_size)
  my_validation_batch_generator = MY_Generator(validate_video_filenames, validate_video_ground_truth, batch_size)


  os.mkdir('/content/drive/My Drive/Affective Computing/Assignment 3/new_chck/')
  # os.mkdir('/content/drive/My Drive/Affective Computing/Assignment 3/logs')
  checkpoint_path = "/content/drive/My Drive/Affective Computing/Assignment 3/new_chck/cp-1.ckpt"
  cp_callback = keras.callbacks.ModelCheckpoint(checkpoint_path,monitor='val_loss',
     verbose=1, save_best_only=True,
     mode = 'min')
  tensorboard = TensorBoard(log_dir="/content/drive/My Drive/Affective Computing/Assignment 3/logs/{}".format(time()))  
  #training started
  model.fit_generator(generator=my_training_batch_generator,
                                         steps_per_epoch=(len(train_video_filenames)//batch_size),
                                         epochs=6,
                                         verbose=1,
                                         validation_data=my_validation_batch_generator,
                                         validation_steps=(len(validate_video_filenames)//batch_size),
                                         callbacks=[tensorboard,cp_callback])
  print("successfully finished training!!!")

main()


1
Epoch 1/6
809/809 [==============================] - 1355s 2s/step - loss: 0.0159 - accuracy: 0.9985 - val_loss: 0.0000e+00 - val_accuracy: 1.0000

Epoch 00001: val_loss improved from inf to 0.00000, saving model to /content/drive/My Drive/Affective Computing/Assignment 3/new_chck/cp-1.ckpt
Epoch 2/6
809/809 [==============================] - 1167s 1s/step - loss: 4.2263e-05 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000

Epoch 00002: val_loss did not improve from 0.00000
Epoch 3/6
809/809 [==============================] - 1161s 1s/step - loss: 1.7496e-06 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000

Epoch 00003: val_loss did not improve from 0.00000
Epoch 4/6
809/809 [==============================] - 1159s 1s/step - loss: 9.5348e-07 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000

Epoch 00004: val_loss did not improve from 0.00000
Epoch 5/6
809/809 [==============================] - 1158s 1s/step - loss: 1.8299e-08 - accuracy:

In [11]:
def decode(prediction):
  for i in range(8):
    if(prediction[0][i] == 1):
      return i
  return -1

def predict():
  model = resnet50_emotion_recoginition()
  model.load_weights('/content/drive/My Drive/Affective Computing/Assignment 3/checkpoints/cp-1.ckpt')
  adam = keras.optimizers.Adam(lr=0.001)
  model.compile(loss=keras.losses.categorical_crossentropy, optimizer=adam, metrics=['accuracy'])
  csv_test_file = pd.read_csv('/content/drive/My Drive/Affective Computing/Assignment 3/my_data/TEST.csv')
  test_z = csv_test_file.iloc[:,1].values
  gt_test = csv_test_file.iloc[:,2].values

  for i in range(500):
    x = load_img(test_z[i])
    x = img_to_array(x)
    x = cv2.resize(x,(224,224))
    video = np.zeros((1,224,224,3))
    video[0,:,:,:] = x[:,:,:]
    video /= 255.
    video -= 1.
    predictions = model.predict(video)
    # print(predictions)
    label = decode(predictions)
    result = ""
    if label == -1:
      result = 'happy'
    elif label == 0:
      result = 'sad'
    elif label == 1:
      result = 'disgust'
    elif label == 2:
      result = 'angry'
    elif label == 3:
      result = 'surprised'
    elif label == 4:
      result = 'calm'
    elif  label ==5:
      result = 'neutral'
    else:
      result = 'fearful'
    print(result,gt_test[i])

predict()

angry angry
angry angry
angry angry
angry angry
angry angry
angry angry
angry angry
angry angry
angry angry
angry angry
angry angry
angry angry
angry angry
angry angry
angry angry
angry angry
angry angry
angry angry
angry angry
angry angry
angry angry
angry angry
angry angry
angry angry
angry angry
angry angry
angry angry
angry angry
angry angry
angry angry
angry angry
angry angry
angry angry
angry angry
angry angry
angry angry
angry angry
angry angry
angry angry
angry angry
angry angry
angry angry
angry angry
angry angry
angry angry
angry angry
angry angry
angry angry
angry angry
angry angry
angry angry
angry angry
angry angry
angry angry
angry angry
angry angry
angry angry
angry angry
angry angry
angry angry
angry angry
angry angry
angry angry
angry angry
angry angry
angry angry
angry angry
angry angry
angry angry
angry angry
angry angry
angry angry
angry angry
angry angry
angry angry
angry angry
angry angry
angry angry
angry angry
angry angry
angry angry
angry angry
angry angry
angr